In [ ]:
import pandas as pd
import csv
import json

**Question**: do uniprotIDs mapped to the same LOC share similar GO terms?

In [ ]:
locs=pd.read_csv("uniprot-pcangsd_all.tab", sep="\t", names=("LOC","uni","1","2","3","4","5","6"), header=None)
locs = locs.iloc[1: , :]

In [ ]:
mapping=pd.read_csv("GO_mapping_topGO", sep="\t", names=("uni","go"))

In [ ]:
mydic={}
for index, row in locs.iterrows():
  loc = row[0]
  same_locs_list=locs[locs["LOC"]==loc]
  if len(same_locs_list) > 1:
    nlist=[]
    for i,l in same_locs_list.iterrows():
      goterms=list(mapping[mapping["uni"]==l[1]].go)
      if len(goterms) != 0: 
        g=goterms[0].split(",")
        nlist.append(g)
    nlist=[item for sublist in nlist for item in sublist]
    nnlist=[x for x in nlist if nlist.count(x)==1]
    if len(nlist) != 0:
      if len(nnlist)/len(nlist) > 0:
        mydic[loc]=(len(nnlist)/len(nlist)) # Number of unique GO terms over all GO terms, over all different matches of LOC to UniprotID

I was right, out of all of the LOCs only a few had unique GOs between the multiple uniprotID hits

So I am just gonna choose the uniprotID with the most GO terms

In [ ]:
newdic={}
for loc in locs.LOC.unique():
  same_locs_list=locs[locs["LOC"]==loc]
  tempdic={}
  for i,l in same_locs_list.iterrows():
    goterms=list(mapping[mapping["uni"]==l[1]].go) # for each of the rows with the same loc, get GO term corresponding to uniprotID
    if len(goterms) != 0: 
      g=goterms[0].split(",")
      tempdic[l[1]] = g

  if len(tempdic) == 0: # no uniprotIDs mapped to any goterms, select first uniprotID
    newdic[loc]=list(same_locs_list.uni)[0]

  else:
    m=0
    k=list(tempdic.keys())[0]
    for key in tempdic:
      num_go=len(tempdic[key])
      if num_go > m:
        m=num_go
        k=key
    newdic[loc]=k



In [ ]:
textfile = open("uniprotIDs_pcangsd_all.txt", "w")

for element in newdic.values():
    textfile.write(element + "\n")
textfile.close()

In [ ]:
#with open('LOC2Uniprot_most_GO.json', 'w') as fp:
    #json.dump(newdic, fp)